In [ ]:
import os
from proteusPy import (
    DisulfideList,
    Disulfide,
    Load_PDB_SS,
    load_disulfides_from_id,
    prune_extra_ss,
    check_header_from_file,
)

from proteusPy.logger_config import get_logger
from proteusPy import (
    extract_ssbonds_and_atoms,
    print_disulfide_bond_info_dict,
    get_residue_atoms_coordinates,
    Initialize_Disulfide_From_Coords,
    get_phipsi_atoms_coordinates,
)

from proteusPy import calc_angle, calc_dihedral
import numpy as np


_logger = get_logger("testing")
# Set the default logger level to CRITICAL
import logging

logging.basicConfig(level=logging.CRITICAL)

In [ ]:
PDB_SS = Load_PDB_SS(verbose=True, subset=False)
PDB_SS.describe()

In [ ]:
pdb_id = "5rsa"
pdb_dir = "/Users/egs/PDB/good"
structure_fname = os.path.join(pdb_dir, f"pdb{pdb_id}.ent")
resolution = -1.0
verbose = False
# model = structure[model_numb]

print(f"-> load_disulfide_from_id() - Parsing structure: {pdb_id}:")

SSList = DisulfideList([], pdb_id, resolution)

ssbond_atom_list, num_ssbonds, errors = extract_ssbonds_and_atoms(
    structure_fname, verbose=verbose, dbg=False
)
print_disulfide_bond_info_dict(ssbond_atom_list)
coords = get_residue_atoms_coordinates(ssbond_atom_list, "A", 26)

In [ ]:
phipsi1 = get_phipsi_atoms_coordinates(ssbond_atom_list, "A", "proximal-1")
phipsi2 = get_phipsi_atoms_coordinates(ssbond_atom_list, "A", "proximal+1")

In [ ]:
print(f"phipsi1: {phipsi1}")
print(f"phipsi2: {phipsi2}")


In [7]:
pdb_id = "5rsa"
pdb_dir = "/Users/egs/PDB/good"
structure_fname = os.path.join(pdb_dir, f"pdb{pdb_id}.ent")
resolution = -1.0
verbose = False

sslist = Initialize_Disulfide_From_Coords(
    ssbond_atom_list, pdb_id, "A", "A", "26", "84", -1.0, verbose=True
)
sslist

<Disulfide 5rsa_26A_84A, Source: 5rsa, Resolution: -1.0 Å>

In [9]:
pdb_id = "5rsa"
pdb_dir = "/Users/egs/PDB/good"
structure_fname = os.path.join(pdb_dir, f"pdb{pdb_id}.ent")
resolution = -1.0
verbose = False

sslist = load_disulfides_from_id(pdb_id, pdb_dir, verbose=True)
sslist

-> load_disulfide_from_id() - Parsing structure: 5rsa:
-> load_disulfides_from_id(): 5rsa has 4 SSBonds, found: 0 errors


DisulfideList([<Disulfide 5rsa_26A_84A, Source: 5rsa, Resolution: -1.0 Å>,
               <Disulfide 5rsa_40A_95A, Source: 5rsa, Resolution: -1.0 Å>,
               <Disulfide 5rsa_58A_110A, Source: 5rsa, Resolution: -1.0 Å>,
               <Disulfide 5rsa_65A_72A, Source: 5rsa, Resolution: -1.0 Å>])

In [ ]:
rsass = DisulfideList([], "5rsa")
rsass = load_disulfides_from_id("5rsa", verbose=True, quiet=True, dbg=True, cutoff=-1.0)
print_disulfide_bond_info_dict()

In [ ]:
rsass